In [83]:
import numpy as np
import pandas as pd
import torch.nn as nn
import os
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from datetime import date, timedelta
import utils
import importlib
import holidays

In [84]:
# load all files into one single df
# df = pd.concat([pd.read_csv('hka-aqm-am/' + f, skiprows=1, sep=';', engine='python') for f in os.listdir('hka-aqm-am/')])
df = pd.concat([pd.read_csv('hka-aqm-am/' + f.removeprefix('._'), skiprows=1, sep=';', engine='python') for f in os.listdir('hka-aqm-am/')])
df.shape


(608036, 18)

In [85]:
df['date_time'] = pd.to_datetime(df['date_time'])
df['weekday'] = df['date_time'].dt.weekday

In [86]:
min_date = df['date_time'].min()
max_date = df['date_time'].max()
print(min_date, max_date)

2022-09-02 12:23:27 2023-10-01 23:51:47


In [87]:
holiday_list = holidays.Germany(years=[2022, 2023, 2024])
exam_dates = holidays.HolidayBase()

# Date ranges for HKA holidays
date_ranges = [
    ('2022-07-25', '2023-09-23'),
    ('2022-12-26', '2023-01-06'),
    ('2023-02-13', '2023-03-12'),
    ('2023-04-11', '2023-04-11'),
    ('2023-05-29', '2023-06-02'),
    ('2023-07-24', '2023-09-22'),
    ('2023-12-22', '2024-01-05'),
    ('2024-02-12', '2024-03-15')
]

# Date ranges for HKA exams
exam_date_ranges = [
    ('2023-01-23', '2023-02-11'),
    ('2023-07-03', '2023-07-21'),
    ('2024-01-22', '2024-02-09')
]

def add_custom_holidays(ranges, holiday_obj):
    for start, end in ranges:
        start_date = date.fromisoformat(start)
        end_date = date.fromisoformat(end)
        current_date = start_date
        while current_date <= end_date:
            holiday_obj.append(current_date)
            current_date += timedelta(days=1)

add_custom_holidays(date_ranges, holiday_list)
add_custom_holidays(exam_date_ranges, exam_dates)


In [88]:
# add feature isHolday and isExamTime
df['isHoliday'] = df['date_time'].dt.date.isin(holiday_list).astype(int)
df['isExamTime'] = df['date_time'].dt.date.isin(exam_dates).astype(int)

In [89]:
df['month'] = df['date_time'].dt.month
df['hour'] = df['date_time'].dt.round('H')
df['semester'] = 'WS22/23'
df.loc[df['date_time'] >= '2023-03-01', 'semester'] = 'SS23'
df.loc[df['date_time'] >= '2023-09-01', 'semester'] = 'WS23/24'
# one hot encoding for semester
df = pd.get_dummies(df, columns=['semester'])


In [90]:
# round date_time by half hour
df['date_time_rounded'] = df['date_time'].dt.round('30min')
# group by device_id and date_time_rounded and mean all columns
df_grouped = df.groupby(['device_id', 'date_time_rounded']).mean().reset_index()

/var/folders/27/rw3093250gvf841xt9hmzs580000gn/T/ipykernel_31181/2168420208.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_grouped = df.groupby(['device_id', 'date_time_rounded']).mean().reset_index()


In [91]:
df.columns

Index(['date_time', 'device_id', 'tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR',
       'WIFI', 'BLE', 'rssi', 'channel_rssi', 'snr', 'gateway',
       'channel_index', 'spreading_factor', 'bandwidth', 'f_cnt', 'weekday',
       'isHoliday', 'isExamTime', 'month', 'hour', 'semester_SS23',
       'semester_WS22/23', 'semester_WS23/24', 'date_time_rounded'],
      dtype='object')

In [92]:
df_grouped.set_index(['device_id', 'date_time_rounded'], inplace=True)

# Sorting the index is important for reindexing and shifting to work correctly
df_grouped.sort_index(inplace=True)

In [93]:
all_times = pd.date_range(df_grouped.index.get_level_values(1).min(), df_grouped.index.get_level_values(1).max(), freq='30T')
multi_index = pd.MultiIndex.from_product([df_grouped.index.get_level_values(0).unique(), all_times], names=['device_id', 'date_time_rounded'])
df_grouped = df_grouped.reindex(multi_index)

In [94]:
df_grouped

tmp  hum  CO2  VOC  vis  IR  WIFI  BLE  \
device_id      date_time_rounded                                             
 hka-aqm-am001 2022-09-02 12:30:00  NaN  NaN  NaN  NaN  NaN NaN   NaN  NaN   
               2022-09-02 13:00:00  NaN  NaN  NaN  NaN  NaN NaN   NaN  NaN   
               2022-09-02 13:30:00  NaN  NaN  NaN  NaN  NaN NaN   NaN  NaN   
               2022-09-02 14:00:00  NaN  NaN  NaN  NaN  NaN NaN   NaN  NaN   
               2022-09-02 14:30:00  NaN  NaN  NaN  NaN  NaN NaN   NaN  NaN   
...                                 ...  ...  ...  ...  ...  ..   ...  ...   
 hka-aqm-am308 2023-10-01 22:00:00  NaN  NaN  NaN  NaN  NaN NaN   NaN  NaN   
               2023-10-01 22:30:00  NaN  NaN  NaN  NaN  NaN NaN   NaN  NaN   
               2023-10-01 23:00:00  NaN  NaN  NaN  NaN  NaN NaN   NaN  NaN   
               2023-10-01 23:30:00  NaN  NaN  NaN  NaN  NaN NaN   NaN  NaN   
               2023-10-02 00:00:00  NaN  NaN  NaN  NaN  NaN NaN   NaN  NaN   

                                    rssi  channel_rssi  ...  spreading_factor  \
device_id      date_time_rounded                        ...                     
 hka-aqm-am001 2022-09-02 12:30:00   NaN           NaN  ...               NaN   
               2022-09-02 13:00:00   NaN           NaN  ...               NaN   
               2022-09-02 13:30:00   NaN           NaN  ...               NaN   
               2022-09-02 14:00:00   NaN           NaN  ...               NaN   
               2022-09-02 14:30:00   NaN           NaN  ...               NaN   
...                                  ...           ...  ...               ...   
 hka-aqm-am308 2023-10-01 22:00:00   NaN           NaN  ...               NaN   
               2023-10-01 22:30:00   NaN           NaN  ...               NaN   
               2023-10-01 23:00:00   NaN           NaN  ...               NaN   
               2023-10-01 23:30:00   NaN           NaN  ...               NaN   
               2023-10-02 00:00:00   NaN           NaN  ...               NaN   

                                    bandwidth  f_cnt  weekday  isHoliday  \
device_id      date_time_rounded                                           
 hka-aqm-am001 2022-09-02 12:30:00        NaN    NaN      NaN        NaN   
               2022-09-02 13:00:00        NaN    NaN      NaN        NaN   
               2022-09-02 13:30:00        NaN    NaN      NaN        NaN   
               2022-09-02 14:00:00        NaN    NaN      NaN        NaN   
               2022-09-02 14:30:00        NaN    NaN      NaN        NaN   
...                                       ...    ...      ...        ...   
 hka-aqm-am308 2023-10-01 22:00:00        NaN    NaN      NaN        NaN   
               2023-10-01 22:30:00        NaN    NaN      NaN        NaN   
               2023-10-01 23:00:00        NaN    NaN      NaN        NaN   
               2023-10-01 23:30:00        NaN    NaN      NaN        NaN   
               2023-10-02 00:00:00        NaN    NaN      NaN        NaN   

                                    isExamTime  month  semester_SS23  \
device_id      date_time_rounded                                       
 hka-aqm-am001 2022-09-02 12:30:00         NaN    NaN            NaN   
               2022-09-02 13:00:00         NaN    NaN            NaN   
               2022-09-02 13:30:00         NaN    NaN            NaN   
               2022-09-02 14:00:00         NaN    NaN            NaN   
               2022-09-02 14:30:00         NaN    NaN            NaN   
...                                        ...    ...            ...   
 hka-aqm-am308 2023-10-01 22:00:00         NaN    NaN            NaN   
               2023-10-01 22:30:00         NaN    NaN            NaN   
               2023-10-01 23:00:00         NaN    NaN            NaN   
               2023-10-01 23:30:00         NaN    NaN            NaN   
               2023-10-02 00:00:00         NaN    NaN            NaN   

                                    semes

In [95]:
lag_features = ['tmp', 'hum', 'CO2', 'VOC', 'vis']
for feature in lag_features:
    for lag in range(1, 6):
        df_grouped[f'{feature}_lag_{lag}'] = df_grouped.groupby(level=0)[feature].shift(lag)
    df_grouped[f'{feature}_next'] = df_grouped.groupby(level=0)[feature].shift(-1)

In [96]:
# Drop rows where all original data columns are NaN (these rows were not present in the original DataFrame)
df_grouped.dropna(subset=lag_features, how='all', inplace=True)

In [97]:
df_grouped

tmp     hum    CO2    VOC    vis    IR  \
device_id      date_time_rounded                                                
 hka-aqm-am001 2022-09-05 13:00:00  25.050  51.890  993.0  606.5   68.0  11.5   
               2022-09-05 13:30:00  25.110  51.935  864.0  497.0  109.0  18.5   
               2022-09-05 14:00:00  25.345  51.760  658.0  491.5   95.5  14.0   
               2022-09-05 14:30:00  25.580  51.630  642.0  579.0  116.0  19.0   
               2022-09-05 16:00:00  25.710  50.270  617.0  633.0   25.0   9.0   
...                                    ...     ...    ...    ...    ...   ...   
 hka-aqm-am308 2023-09-26 22:00:00  27.435  37.415  522.5  957.5    4.0   1.0   
               2023-09-26 22:30:00  27.380  37.510  527.5  958.5    5.5   2.0   
               2023-09-26 23:00:00  27.370  37.580  524.5  949.0    5.5   2.5   
               2023-09-26 23:30:00  27.345  37.630  527.0  955.0    4.0   0.0   
               2023-09-27 00:00:00  27.330  37.650  525.0  951.0    4.0   0.0   

                                    WIFI   BLE   rssi  channel_rssi  ...  \
device_id      date_time_rounded                                     ...   
 hka-aqm-am001 2022-09-05 13:00:00   1.0   0.5 -127.5        -127.5  ...   
               2022-09-05 13:30:00   2.0   0.0 -128.5        -128.5  ...   
               2022-09-05 14:00:00   2.5   0.5 -134.0        -134.0  ...   
               2022-09-05 14:30:00   0.0   0.0 -134.0        -134.0  ...   
               2022-09-05 16:00:00   1.0   0.0 -138.0        -138.0  ...   
...                                  ...   ...    ...           ...  ...   
 hka-aqm-am308 2023-09-26 22:00:00   3.5  13.5  -74.0         -74.0  ...   
               2023-09-26 22:30:00   1.0   0.0  -73.0         -73.0  ...   
               2023-09-26 23:00:00   1.5   0.0  -72.5         -72.5  ...   
               2023-09-26 23:30:00   3.0  13.5  -75.0         -75.0  ...   
               2023-09-27 00:00:00   0.0   0.0  -70.0         -70.0  ...   

                                    VOC_lag_3  VOC_lag_4  VOC_lag_5  VOC_next  \
device_id      date_time_rounded                                                
 hka-aqm-am001 2022-09-05 13:00:00        NaN        NaN        NaN     497.0   
               2022-09-05 13:30:00        NaN        NaN        NaN     491.5   
               2022-09-05 14:00:00        NaN        NaN        NaN     579.0   
               2022-09-05 14:30:00      606.5        NaN        NaN       NaN   
               2022-09-05 16:00:00      579.0      491.5      497.0       NaN   
...                                       ...        ...        ...       ...   
 hka-aqm-am308 2023-09-26 22:00:00      953.0      953.5      944.0     958.5   
               2023-09-26 22:30:00      950.5      953.0      953.5     949.0   
               2023-09-26 23:00:00      955.0      950.5      953.0     955.0   
               2023-09-26 23:30:00      957.5      955.0      950.5     951.0   
               2023-09-27 00:00:00      958.5      957.5      955.0       NaN   

                                    vis_lag_1  vis_lag_2  vis_lag_3  \
device_id      date_time_rounded                                      
 hka-aqm-am001 2022-09-05 13:00:00        NaN        NaN        NaN   
               2022-09-05 13:30:00       68.0        NaN        NaN   
               2022-09-05 14:00:00      109.0       68.0        NaN   
               2022-09-05 14:30:00       95.5      109.0       68.0   
               2022-09-05 16:00:00        NaN        NaN      116.0   
...                                       ...        ...        ...   
 hka-aqm-am308 2023-09-26 22:00:00        4.0        5.5        5.5   
               2023-09-26 22:30:00        4.0        4.0        5.5   
               2023-09-26 23:00:00        5.5        4.0        4.0   
               2023-09-26 23:30:00        5.5        5.5        4.0   
               2023-09-27 00:00:00        4.0        5.5        5.5   

                  

In [100]:
df_grouped.columns

Index(['tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR', 'WIFI', 'BLE', 'rssi',
       'channel_rssi', 'channel_index', 'spreading_factor', 'bandwidth',
       'f_cnt', 'weekday', 'isHoliday', 'isExamTime', 'month', 'semester_SS23',
       'semester_WS22/23', 'semester_WS23/24', 'tmp_lag_1', 'tmp_lag_2',
       'tmp_lag_3', 'tmp_lag_4', 'tmp_lag_5', 'tmp_next', 'hum_lag_1',
       'hum_lag_2', 'hum_lag_3', 'hum_lag_4', 'hum_lag_5', 'hum_next',
       'CO2_lag_1', 'CO2_lag_2', 'CO2_lag_3', 'CO2_lag_4', 'CO2_lag_5',
       'CO2_next', 'VOC_lag_1', 'VOC_lag_2', 'VOC_lag_3', 'VOC_lag_4',
       'VOC_lag_5', 'VOC_next', 'vis_lag_1', 'vis_lag_2', 'vis_lag_3',
       'vis_lag_4', 'vis_lag_5', 'vis_next'],
      dtype='object')